In [25]:
import pandas as pd

## Задание 1

In [26]:
log = pd.read_csv('visit_log.csv', sep = ';')

log['source_type'] = log['traffic_source']      ## Сначала заполняем значения по умолчанию

log.loc[
        (log['traffic_source'].str.lower() == 'yandex') | (log['traffic_source'].str.lower() == 'google'),      ## Значения по yandex и google
        'source_type'
] = 'organic'

log.loc[
        (log['traffic_source'].str.lower() == 'paid') | (log['traffic_source'].str.lower() == 'email'),         ## Значения по yandex и google без России
        'source_type'
] = 'other'

log.loc[
        (
            (log['traffic_source'].str.lower() == 'paid') | (log['traffic_source'].str.lower() == 'email')      ## Значения по yandex и google с Россией
        )            
        &
        (
            log['region'].str.lower() == 'russia'
        ), 
        'source_type'
] = 'ad'

log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


## Задание 2

In [27]:
df = pd.read_csv('URLs.txt')
news_urls = df.loc[df['url'].str.contains(pat = r'/\d{8}-', regex = True)]
news_urls.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3

In [28]:
ratings = pd.read_csv('../HW2_Pandas/ml-25m/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [29]:
user_stat = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': ['min', 'max']})     ## Для каждого пользователя считаем кол-во оценок, минимальное и максимальное время оценки
user_stat.loc[
    (user_stat['rating']['count'] > 100),                                                       ## Выбираем пользователей с более чем 100 оценками
    'lifetime'                                                                                  ## Новому столбцу lifetime
] = user_stat['timestamp']['max'] - user_stat['timestamp']['min']                               ## присваиваем значение кол-ва секунд м/у максимальным и минимальным временем оценки
user_stat.head()

rating   timestamp                 lifetime
        count         min         max             
userId                                            
1          70  1147868053  1147880055          NaN
2         184  1141415509  1141418008       2499.0
3         656  1439472199  1566091883  126619684.0
4         242  1573937091  1574195101     258010.0
5         101   830786155   858628421   27842266.0

## Задание 4

In [30]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [31]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [32]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [33]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [34]:
## Все значения для client_id уникальны => доп. обработка таблиц не требуется
revenue1 = rzd.merge(right = auto, on = 'client_id', how = 'outer').merge(right = air, on = 'client_id', how = 'outer').fillna(0)   ## Объединаем все таблицы с выручками в одну
revenue1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [35]:
revenue2 = client_base.merge(right = revenue1, on = 'client_id', how = 'inner')                                                      ## Добавляем таблицу с адресами
revenue2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
